In [3]:
pip install pandas

  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sampr\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import pandas as pd

df = pd.read_csv(r'C:\Users\sampr\OneDrive\Documents\zenflix\data\raw\Movie_Data_File.csv')
df.head()
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Film_title         10002 non-null  object 
 1   Release_year       0 non-null      float64
 2   Director           9992 non-null   object 
 3   Cast               9945 non-null   object 
 4   Average_rating     9421 non-null   float64
 5   Owner_rating       967 non-null    float64
 6   Genres             9978 non-null   object 
 7   Runtime            9992 non-null   float64
 8   Countries          9884 non-null   object 
 9   Original_language  9999 non-null   object 
 10  Spoken_languages   9999 non-null   object 
 11  Description        10000 non-null  object 
 12  Studios            9695 non-null   object 
 13  Watches            10002 non-null  int64  
 14  List_appearances   10002 non-null  int64  
 15  Likes              10002 non-null  int64  
 16  Fans               100

Film_title               0
Release_year         10002
Director                10
Cast                    57
Average_rating         581
Owner_rating          9035
Genres                  24
Runtime                 10
Countries              118
Original_language        3
Spoken_languages         3
Description              2
Studios                307
Watches                  0
List_appearances         0
Likes                    0
Fans                     0
½                        0
★                        0
★½                       0
★★                       0
★★½                      0
★★★                      0
★★★½                     0
★★★★                     0
★★★★½                    0
★★★★★                    0
Total_ratings            0
Film_URL                 0
dtype: int64

In [41]:
df.drop(columns=['Release_year','Owner_rating'], inplace=True, errors='ignore')


renaming columns

In [42]:
df.rename(columns={
    'Film_title': 'title',
    'Director': 'director',
    'Cast': 'cast',
    'Average_rating': 'average_rating',
    'Genres': 'genres',
    'Runtime': 'runtime',
    'Countries': 'countries',
    'Original_language': 'language',
    'Spoken_languages': 'spoken_languages',
    'Description': 'description',
    'Studios': 'studio',
    'Watches': 'watch_count',
    'List_appearances': 'list_appearances',
    'Likes': 'likes',
    'Fans': 'fans',
    '½': 'half_star_count',
    '★': '1_star_count',
    '★½': '1_5_star_count',
    '★★': '2_star_count',
    '★★½': '2_5_star_count',
    '★★★': '3_star_count',
    '★★★½': '3_5_star_count',
    '★★★★': '4_star_count',
    '★★★★½': '4_5_star_count',
    '★★★★★': '5_star_count',
    'Total_ratings': 'total_ratings',
    'Film_URL': 'film_url'
}, inplace=True)


In [44]:
df.columns

Index(['title', 'director', 'cast', 'average_rating', 'owner_rating', 'genres',
       'runtime', 'countries', 'language', 'spoken_languages', 'description',
       'studio', 'watch_count', 'list_appearances', 'likes', 'fans',
       'half_star_count', '1_star_count', '1_5_star_count', '2_star_count',
       '2_5_star_count', '3_star_count', '3_5_star_count', '4_star_count',
       '4_5_star_count', '5_star_count', 'total_ratings', 'film_url',
       'clean_description', 'clean_genres', 'clean_cast', 'clean_studio',
       'genres_list', 'all_text', 'sentiment_polarity', 'avg_rating_norm',
       'runtime_norm', 'votes_norm'],
      dtype='object')

In [16]:
df.dropna(subset=['title', 'description', 'genres', 'average_rating'], inplace=True)
df.drop_duplicates(subset='title', inplace=True)


 Clean Text Fields (for NLP use)
Clean description, keywords, genres for mood-mapping and similarity matching.

In [24]:
import re

def clean_text(text):
    if pd.isnull(text): return ""
    text = re.sub(r"[^a-zA-Z0-9\s]", "", str(text))  # remove special characters
    return text.lower()

df['clean_description'] = df['description'].apply(clean_text)
df['clean_genres'] = df['genres'].apply(clean_text)
df['clean_cast'] = df['cast'].apply(clean_text)
df['clean_studio'] = df['studio'].apply(clean_text)


converting genre to list for modelling:

In [25]:
df['genres_list'] = df['genres'].apply(lambda x: [i.strip() for i in str(x).split(",")])


In [19]:
df['all_text'] = (
    df['clean_description'] + ' ' +
    df['clean_genres'] + ' ' +
    df['clean_cast'] + ' ' +
    df['clean_studio']
)


here we created a composite nlp field all_text column which will feed into:

TF-IDF vectorizers

Cosine similarity

Mood-mapping models

Sentiment analyzers



In [20]:
df = df[df['average_rating'] > 1.0]
df = df[df['runtime'].notnull() & (df['runtime'] > 20)]  # likely real movies


standardaizing genre and language entries

In [26]:
def standardize_genres(genres):
    mapping = {
        "sci fi": "science fiction",
        "sci-fi": "science fiction",
        "romcom": "romantic comedy",
        "dramedy": "drama comedy"
    }
    genres = [mapping.get(g.strip(), g.strip()) for g in genres]
    return list(set(genres))  # remove duplicates

df['genres_list'] = df['genres_list'].apply(standardize_genres)


In [27]:
df['language'] = df['language'].str.lower().str.strip()


2. Extract Mood-Associated Keywords (for NLP models)
Later, for emotion-to-genre or “Cinematic Zodiac” use, you'll want to extract top keywords or phrases from the description.

python
Copy code


In [29]:
pip install scikit-learn


  Using cached scikit_learn-1.6.1-cp39-cp39-win_amd64.whl (11.2 MB)
  Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sampr\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
X = vectorizer.fit_transform(df['clean_description'])
tfidf_features = vectorizer.get_feature_names_out()


3. Director Popularity Score (for Leaderboards)
Useful for "Letterboxd Royals" and hidden gem discovery. stored separately for dashboards

In [31]:
director_stats = df.groupby('director').agg({
    'title': 'count',
    'average_rating': 'mean',
    'total_ratings': 'sum'
}).reset_index()

director_stats.rename(columns={
    'title': 'num_movies',
    'average_rating': 'avg_rating',
    'total_ratings': 'total_votes'
}, inplace=True)

director_stats['popularity_score'] = (
    director_stats['avg_rating'] * director_stats['num_movies']
)


4. Generate Sentiment Score for Each Movie (for mood mapping)
This enables mood-based movie therapy, "Which Movie Are You", and "EmoGraph". gives a score bw -1 and 1 used to tag moods like Reflective Heartbroken Uplifting and Anxious

In [33]:
pip install textblob


  Using cached click-8.1.8-py3-none-any.whl (98 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sampr\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [34]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sampr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [35]:
from textblob import TextBlob

df['sentiment_polarity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.polarity)


Normalize Numeric Features
If you're building ML models like in CineClash or EmoGraph:

Normalize average_rating, runtime, total_ratings, etc.

In [36]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['avg_rating_norm', 'runtime_norm', 'votes_norm']] = scaler.fit_transform(
    df[['average_rating', 'runtime', 'total_ratings']]
)


saving as new file 

In [37]:
df.to_csv('../data/processed/movies_cleaned.csv', index=False)
director_stats.to_csv('../data/processed/director_stats.csv', index=False)


printing all the unique genres in the ds (useful for recs)

In [ ]:
import pandas as pd


df = pd.read_csv(r'C:\Users\sampr\OneDrive\Documents\zenflix\data\processed\movies_cleaned.csv')

# Drop missing or empty genre entries
df = df.dropna(subset=['genres'])

# Split genres by commas and flatten the list
genre_lists = df['genres'].apply(lambda x: [genre.strip() for genre in x.split(',')])
all_genres = set([genre for sublist in genre_lists for genre in sublist])

# Display all unique genres in the dataset
print("Unique genres in dataset:")
for genre in sorted(all_genres):
    print(f"- {genre}")


Unique genres in dataset:
- 'Action'
- 'Action']
- 'Adventure'
- 'Adventure']
- 'Animation'
- 'Animation']
- 'Captivating vision and Shakespearean drama'
- 'Comedy'
- 'Comedy']
- 'Crime'
- 'Crime']
- 'Documentary'
- 'Documentary']
- 'Drama'
- 'Drama']
- 'Epic history and literature'
- 'Family'
- 'Family']
- 'Fantasy adventure
- 'Fantasy'
- 'Fantasy']
- 'Historical battles and epic heroism'
- 'History'
- 'History']
- 'Horror'
- 'Horror']
- 'Music'
- 'Music']
- 'Mystery'
- 'Mystery']
- 'Romance'
- 'Romance']
- 'Science Fiction'
- 'Science Fiction']
- 'Show All…']
- 'TV Movie'
- 'TV Movie']
- 'Thriller'
- 'Thriller']
- 'War'
- 'War']
- 'Western'
- 'Western']
- ['Action'
- ['Action']
- ['Adventure'
- ['Adventure']
- ['Animation'
- ['Animation']
- ['Comedy'
- ['Comedy']
- ['Crime'
- ['Crime']
- ['Documentary'
- ['Documentary']
- ['Drama'
- ['Drama']
- ['Epic heroes'
- ['Family'
- ['Family']
- ['Fantasy'
- ['Fantasy']
- ['History'
- ['History']
- ['Horror'
- ['Horror']
- ['Music'
- ['Music']

In [51]:
import re

def clean_genre_text(genre_str):
    # Remove brackets, quotes, etc.
    genre_str = re.sub(r"[\[\]\'\"]", "", genre_str)
    genres = [g.strip() for g in genre_str.split(',')]
    # Keep only genres that are short (1-2 words)
    cleaned = [g for g in genres if len(g.split()) <= 2 and len(g) > 2]
    return cleaned

# Apply cleaning
genre_lists = df['genres'].dropna().apply(clean_genre_text)

# Flatten the genre list
all_genres = set([genre for sublist in genre_lists for genre in sublist])

# Print cleaned, unique genres
print("Cleaned Unique Genres in Dataset:")
for genre in sorted(all_genres):
    print(f"- {genre}")


Cleaned Unique Genres in Dataset:
- Action
- Adventure
- Animation
- Comedy
- Crime
- Documentary
- Drama
- Epic heroes
- Family
- Fantasy
- Fantasy adventure
- History
- Horror
- Music
- Mystery
- Romance
- Science Fiction
- Show All…
- TV Movie
- Thriller
- War
- Western
- and swordplay
- heroism


In [54]:
# Save the cleaned dataset
df.to_csv(r'C:\Users\sampr\OneDrive\Documents\zenflix\data\processed/movies_cleaned_final.csv', index=False)


In [57]:
df.columns

Index(['title', 'director', 'cast', 'average_rating', 'owner_rating', 'genres',
       'runtime', 'countries', 'language', 'spoken_languages', 'description',
       'studio', 'watch_count', 'list_appearances', 'likes', 'fans',
       'half_star_count', '1_star_count', '1_5_star_count', '2_star_count',
       '2_5_star_count', '3_star_count', '3_5_star_count', '4_star_count',
       '4_5_star_count', '5_star_count', 'total_ratings', 'film_url',
       'clean_description', 'clean_genres', 'clean_cast', 'clean_studio',
       'genres_list', 'all_text', 'sentiment_polarity', 'avg_rating_norm',
       'runtime_norm', 'votes_norm'],
      dtype='object')

TESTING FOR RECOMMENDATION SYSTEM

In [ ]:
import sys
import os


sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from recommender.mood_based import recommend_movies_by_mood


recommend_movies_by_mood("Happy")


[{'Title': 'Smashed',
  'Runtime': 85.0,
  'Director': 'James Ponsoldt',
  'Cast': "['Mary Elizabeth Winstead', 'Aaron Paul', 'Octavia Spencer', 'Nick Offerman', 'Megan Mullally', 'Mary Kay Place', 'Kyle Gallner', 'Mackenzie Davis', 'Bree Turner', 'Brad Carter', 'Barrett Shuler', 'Rene Rivera', 'Richmond Arquette', 'David Grammer', 'Ron Lynch', 'Elise Salomon', 'Anjini Taneja Azhar', 'Aileen Davila', 'Silas Agape Garcia', 'Wilson Schwartz', 'Haley Brooke Walker', 'Lisa Yamada', 'Murray Gershenz', 'Jane Noltemeyer', 'Brian C. Haynes', 'Patti Allison', 'Gabriel Aslan', 'Carol Avery', 'Susan Burke', 'Danilo Di Julio', 'Natalie Dreyfuss', 'Kelly Ebsary', 'Daniel Jay Salmen']",
  'Genre': "['Romance', 'Comedy', 'Drama']",
  'Language': 'english',
  'Description': 'Kate and Charlie like to have a good time. Their marriage thrives on a shared fondness for music, laughter… and getting smashed. When Kate’s partying spirals into hard-core asocial behavior, compromising her job as an elementary s

In [ ]:
import sys
import os

# Add the src directory to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from recommender.mood_based import recommend_movies_by_mood

# List of moods you want to test
moods = ["Happy", "Sad", "Angry", "Anxious", "Bored"]


for mood in moods:
    print(f"Recommendations for mood: {mood}")
    recommended_movies = recommend_movies_by_mood(mood)
    
    # Check if movies are returned for each mood
    if recommended_movies:
        for movie in recommended_movies:
            print(f"Title: {movie['Title']}")
            print(f"Director: {movie['Director']}")
            print(f"Cast: {movie['Cast']}")
            print(f"Runtime: {movie['Runtime']}")
            print(f"Genre: {movie['Genre']}")
            print(f"Language: {movie['Language']}")
            print(f"Description: {movie['Description']}")
            print("="*50)
    else:
        print("No recommendations available for this mood.")
    print("="*100)


Recommendations for mood: Happy
Title: The Kid & I
Director: Penelope Spheeris
Cast: ['Tom Arnold', 'Eric Gores', 'Linda Hamilton', 'Joe Mantegna', 'Henry Winkler', 'Richard Edson', 'Shannon Elizabeth', 'Brenda Strong', 'Arielle Kebbel', "Shaquille O'Neal", 'Bill Goldberg', "Pat O'Brien", 'Eric Dickerson', 'Charles Napier', 'Lisa Wilhoit', 'Melissa Price', 'Brianne Davis', 'Melissa Steach', 'Jamie Lee Curtis', 'Penelope Spheeris', 'Arnold Schwarzenegger']
Runtime: 93.0
Genre: ['Comedy', 'Family']
Language: english
Description: Aaron Roman is a teenager with cerebral palsy who dreams of starring in a big-time action movie. When his father grants Aaron his wish for his eighteenth birthday, he experiences the reality a bit hard to manage.
Title: Gamera vs. Gyaos
Director: Noriaki Yuasa
Cast: ['Kōjirō Hongō', 'Kichijirō Ueda', 'Reiko Kasahara', 'Naoyuki Abe', 'Taro Marui', 'Yukitaro Hotaru', 'Yoshirō Kitahara', 'Fujio Murakami', 'Shō Natsuki', 'Kenji Oyama', 'Kōichi Itō', 'Teppei Endō', 'S

okay so here notice in the print functions we use capital letters for title, cast genre and so on , that is bc we used the append func as capital letters in the main pood based python file this 

TESTING FOR CINECLASH 

In [ ]:
import sys
import os
# Add the src directory to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from recommender.cineclash import compare_movies

result = compare_movies("The Fan", "Mad Max: Fury Road")

print("=== CineClash Results ===")
print(f"{result['Movie 1']['Title']} → Score: {result['Movie 1']['Score']}")
print(f"{result['Movie 2']['Title']} → Score: {result['Movie 2']['Score']}")
print(f"\n🏆 Winner: {result['Winner']}")


=== CineClash Results ===
The Fan → Score: 2452.26
Mad Max: Fury Road → Score: 474850.49

🏆 Winner: Mad Max: Fury Road


TESTING FOR LEADERBOARD

In [1]:
import sys
sys.path.append('../src')

from leaderboard.director_popularity import (
    load_director_stats,
    get_top_directors,
    get_underrated_legends,
    get_most_prolific,
    get_most_viewed_directors,
    get_popular_and_critically_acclaimed
)

df = load_director_stats()

if df.empty:
    print("Failed to load director stats.")
else:
    print("🎬 Top Directors:")
    print(get_top_directors(df), "\n")

    print("🌟 Underrated Legends:")
    print(get_underrated_legends(df), "\n")

    print("🎥 Most Prolific Directors:")
    print(get_most_prolific(df), "\n")

    print("🔥 Most Viewed Directors (by total_votes):")
    print(get_most_viewed_directors(df), "\n")

    print("🏆 Popular & Critically Acclaimed:")
    print(get_popular_and_critically_acclaimed(df), "\n")


🎬 Top Directors:
              director  num_movies  avg_rating  total_votes  popularity_score
45      Akira Kurosawa           7    4.420000      1133636             30.94
2705  Masaki Kobayashi           3    4.396667       213437             13.19
1595    Ingmar Bergman           9    4.330000      1134900             38.97
169   Andrei Tarkovsky           7    4.311429       872092             30.18
521          Béla Tarr           3    4.296667        97927             12.89 

🌟 Underrated Legends:
                              director  num_movies  avg_rating  total_votes  \
847                      David Barnard           1        4.71        42405   
1093  Elizabeth White, Justin Anderson           1        4.63        38915   
2964                         Nate Cash           1        4.62       300121   
1070                       Edward Yang           1        4.54       107185   
1844                 Jennie Livingston           1        4.52       157173   

      popularity

TESTING FOR ZODIAC

In [4]:
import sys
sys.path.append('../src')
from zodiac.zodiac_generator import load_movie_data, assign_zodiac_roles
df = load_movie_data()
df.head()  # Optional: just to inspect the structure


,title,director,cast,average_rating,owner_rating,genres,runtime,countries,language,spoken_languages,...,clean_description,clean_genres,clean_cast,clean_studio,genres_list,all_text,sentiment_polarity,avg_rating_norm,runtime_norm,votes_norm
0,The Fan,Eckhart Schmidt,"['Désirée Nosbusch', 'Bodo Staiger', 'Simone B...",3.57,NaN,"['Horror', 'Drama']",92.0,['Germany'],german,['German'],...,the heroworship that simone has for a pop sing...,horror drama,dsire nosbusch bodo staiger simone brahmann jo...,barbara moorse workshop,"[""['Horror'"", ""'Drama']""]",the heroworship that simone has for a pop sing...,-0.237500,0.685950,0.080135,0.002161
1,Mad Max: Fury Road,George Miller,"['Tom Hardy', 'Charlize Theron', 'Nicholas Hou...",4.18,4.5,"['Adventure', 'Science Fiction', 'Action']",121.0,"['Australia', 'USA']",english,['English'],...,an apocalyptic story set in the furthest reach...,adventure science fiction action,tom hardy charlize theron nicholas hoult hugh ...,warner bros pictures village roadshow pictures...,"[""'Science Fiction'"", ""['Adventure'"", ""'Action...",an apocalyptic story set in the furthest reach...,0.100000,0.853994,0.112867,0.411209
2,Suspiria,Dario Argento,"['Jessica Harper', 'Stefania Casini', 'Flavio ...",3.93,4.0,['Horror'],99.0,['Italy'],english,"['English', 'French', 'German', 'Italian', 'La...",...,an american newcomer to a prestigious german b...,horror,jessica harper stefania casini flavio bucci mi...,seda spettacoli,"[""['Horror']""]",an american newcomer to a prestigious german b...,-0.166667,0.785124,0.088036,0.108427
3,Lost in Translation,Sofia Coppola,"['Bill Murray', 'Scarlett Johansson', 'Akiko T...",3.79,4.5,"['Drama', 'Comedy', 'Romance']",102.0,"['UK', 'USA']",english,"['English', 'Japanese']",...,two lost souls visiting tokyo the young negle...,drama comedy romance,bill murray scarlett johansson akiko takeshita...,american zoetrope elemental films,"[""'Romance']"", ""['Drama'"", ""'Comedy'""]",two lost souls visiting tokyo the young negle...,0.001667,0.746556,0.091422,0.263210
4,Akira,Katsuhiro Otomo,"['Mitsuo Iwata', 'Nozomu Sasaki', 'Mami Koyama...",4.28,5.0,"['Animation', 'Action', 'Science Fiction']",124.0,['Japan'],japanese,['Japanese'],...,a secret military project endangers neotokyo w...,animation action science fiction,mitsuo iwata nozomu sasaki mami koyama tar ish...,mbs sumitomo corporation toho studio fuga stud...,"[""'Action'"", ""'Science Fiction']"", ""['Animatio...",a secret military project endangers neotokyo w...,-0.166667,0.881543,0.116253,0.146797


In [5]:
favorite_movies = [
    "Inception",  # Sun Movie
    "Eternal Sunshine of the Spotless Mind",  # Moon Movie
    "The Grand Budapest Hotel"  # Rising Movie
]
zodiac_chart = assign_zodiac_roles(df, favorite_movies)
print("\n🎬 Your Cinematic Zodiac Chart:")
for role, details in zodiac_chart.items():
    print(f"\n🪐 {role} Movie:")
    if isinstance(details, str):
        print(f"  ⚠️ {details}")
    else:
        print(f"  Your Pick: {details['Your Pick']}")
        print(f"  Matched Movie: {details['Zodiac Match']}")
        print(f"  Genre Similarity: {details['Genre Similarity']}")
        print(f"  Sentiment Polarity Match: {details['Sentiment Polarity Match']}")
        print(f"  Director: {details['Director']}")
        print(f"  Genre: {details['Genre']}")
        print(f"  Description: {details['Description'][:150]}...")



🎬 Your Cinematic Zodiac Chart:

🪐 Sun Movie:
  Your Pick: Inception
  Matched Movie: Mad Max: Fury Road
  Genre Similarity: 1.0
  Sentiment Polarity Match: 0.1
  Director: George Miller
  Genre: ['Adventure', 'Science Fiction', 'Action']
  Description: An apocalyptic story set in the furthest reaches of our planet, in a stark desert landscape where humanity is broken, and most everyone is crazed figh...

🪐 Moon Movie:
  Your Pick: Eternal Sunshine of the Spotless Mind
  Matched Movie: Her
  Genre Similarity: 1.0
  Sentiment Polarity Match: 0.156
  Director: Spike Jonze
  Genre: ['Science Fiction', 'Drama', 'Romance']
  Description: In the not so distant future, Theodore, a lonely writer, purchases a newly developed operating system designed to meet the user's every need. To Theod...

🪐 Rising Movie:
  Your Pick: The Grand Budapest Hotel
  Matched Movie: The Truman Show
  Genre Similarity: 1.0
  Sentiment Polarity Match: 0.176
  Director: Peter Weir
  Genre: ['Comedy', 'Drama']
  Descr

as you can see from the above output we have a genre similarity score as well as a polarity score. the polarity score is obtained from a NLP fetaure called textblob which analyses the sentiment and give a score from -1 to 1 . since the users dont have an idea about what the scores could mean for thier sun moon and rising, ive decided to change up the coe a little so as to give an explanation for each logic step in the output.

new code w explanation: 

In [ ]:

import sys
sys.path.append('../src')
from zodiac.zodiac_generator import assign_zodiac_roles, load_movie_data
import pandas as pd


# Sample user movie picks
user_movies = ["Inception", "The Notebook", "The Dark Knight"]


movie_data = load_movie_data(r'C:\Users\sampr\OneDrive\Documents\zenflix\data\processed\movies_cleaned_final.csv')

zodiac_results = assign_zodiac_roles(movie_data, user_movies)


for role, details in zodiac_results.items():
    print(f"Zodiac Role: {role}")
    print(f"Your Pick: {details['Your Pick']}")
    print(f"Zodiac Match: {details['Zodiac Match']}")
    print(f"Genre Similarity: {details['Genre Similarity']}")
    print(f"Sentiment Polarity Score: {details['Sentiment Polarity Score']}")
    print(f"Reason for Genre Similarity: {details['Reason for Genre Similarity']}")
    print(f"Reason for Sentiment Match: {details['Reason for Sentiment Match']}")
    print(f"Zodiac Role Explanation: {details['Zodiac Role']}")
    print(f"Director: {details['Director']}")
    print(f"Genre: {details['Genre']}")
    print(f"Description: {details['Description']}")
    print('-' * 50)


Zodiac Role: Sun
Your Pick: Inception
Zodiac Match: Mad Max: Fury Road
Genre Similarity: 1.0
Sentiment Polarity Score: 0.0999999999999999
Reason for Genre Similarity: The genre similarity score of 1.0 indicates that the genres of Inception and Mad Max: Fury Road are highly aligned.
Reason for Sentiment Match: The sentiment polarity score of 0.0999999999999999 indicates a positive sentiment, reflecting your emotional connection to the movie.
Zodiac Role Explanation: Your Sun movie is Mad Max: Fury Road because it aligns with your sun traits.
Director: George Miller
Genre: ['Adventure', 'Science Fiction', 'Action']
Description: An apocalyptic story set in the furthest reaches of our planet, in a stark desert landscape where humanity is broken, and most everyone is crazed fighting for the necessities of life. Within this world exist two rebels on the run who just might be able to restore order.
--------------------------------------------------
Zodiac Role: Moon
Your Pick: The Notebook
Zo

TESTING FOR actor based recommendation 

In [ ]:
# test_actor_insights.py  
import sys
sys.path.append('../src')
from recommender.actor_insights import generate_actor_insights, load_movies_data

path = r'C:\Users\sampr\OneDrive\Documents\zenflix\data\processed\movies_cleaned_final.csv' 


df = load_movies_data(path)

# Get actor insights
actor_insights = generate_actor_insights(df, top_n=5)

# Print 
for actor_data in actor_insights:
    print(f"🎭 Actor: {actor_data['actor']} ({actor_data['appearances']} appearances)")
    print(f"❤️ Most Liked Movie: {actor_data['top_liked_movie']['title']} - {actor_data['top_liked_movie']['genre']}")
    print(f"   ➤ {actor_data['top_liked_movie']['description']}")
    print(f"👁️ Most Watched Movie: {actor_data['top_watched_movie']['title']} - {actor_data['top_watched_movie']['genre']}")
    print(f"   ➤ {actor_data['top_watched_movie']['description']}")


🎭 Actor: Frank Welker (100 appearances)
❤️ Most Liked Movie: The Lion King - ['Animation', 'Family', 'Drama']
   ➤ Young lion prince Simba, eager to one day become king of the Pride Lands, grows up under the watchful eye of his father Mufasa; all the while his villainous uncle Scar conspires to take the throne for himself. Amid betrayal and tragedy, Simba must confront his past and find his rightful place in the Circle of Life.
👁️ Most Watched Movie: The Lion King - ['Animation', 'Family', 'Drama']
   ➤ Young lion prince Simba, eager to one day become king of the Pride Lands, grows up under the watchful eye of his father Mufasa; all the while his villainous uncle Scar conspires to take the throne for himself. Amid betrayal and tragedy, Simba must confront his past and find his rightful place in the Circle of Life.
🎭 Actor: Samuel L. Jackson (72 appearances)
❤️ Most Liked Movie: Pulp Fiction - ['Crime', 'Thriller']
   ➤ A burger-loving hit man, his philosophical partner, a drug-addled g

TESTING FOR HIDDEN GEMS AND TOP 5 MOVIES OF EACH SPECIFIC GENRE (added from the leaderboard module)

In [ ]:
import pandas as pd
import sys
sys.path.append('../src')
from leaderboard.director_popularity import get_hidden_gems, get_top_movies_by_genre  


df = pd.read_csv(r'C:\Users\sampr\OneDrive\Documents\zenflix\data\processed\movies_cleaned_final.csv')

hidden_gems = get_hidden_gems(df, max_watch_count=5000, min_rating=4.0, top_n=10)

# Outputting the result as a clean table
print("🎯 Hidden Gems Leaderboard:\n")
print(hidden_gems.to_string(index=False))  # Prints without row numbers



# Test: Top Movies by Genre
genre_input = "Drama"  # Change this to test different genres
print(f"\n🎬 Top Movies in Genre: {genre_input}")
top_genre_movies = get_top_movies_by_genre(df, genre_input, top_n=5)
print(top_genre_movies.to_string(index=False))  # No row numbers


🎯 Hidden Gems Leaderboard:

                                                            title                      director language                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

TESTING FOR PERSONALITY QUIZ

In [ ]:


import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import sys
sys.path.append('../src')
from personality_matcher.personality_quiz import load_movies, get_quiz_questions, calculate_sentiment_score, match_movie_by_sentiment, run_personality_quiz


# Test the loading of the dataset
df = load_movies(path=r"C:/Users/sampr/OneDrive/Documents/zenflix/data/processed/movies_cleaned_final.csv")

# Test the quiz questions and options
questions, options = get_quiz_questions()
for question, option in zip(questions, options):
    print(question)
    for opt in option:
        print(f"   {opt}")

# Test sentiment calculation
user_answers = [1, 3, 4, 2, 5]  # Mock answers to quiz questions
sentiment_score = calculate_sentiment_score(user_answers)
print(f"Sentiment Score: {sentiment_score}")  # Should print a score between -1 and 1

# Full test run of the quiz
full_quiz_result = run_personality_quiz(user_answers)
print(full_quiz_result)


1. How do you usually handle conflict?
   1. Calm and calculated
   2. Direct and bold
   3. Quietly reflective
   4. Avoid at all costs
   5. Emotional but fair
2. What's your ideal weekend like?
   1. Reading a book
   2. Partying with friends
   3. Exploring nature
   4. Binging shows
   5. Creating something new
3. Which trait best defines you?
   1. Loyal
   2. Ambitious
   3. Dreamy
   4. Humble
   5. Passionate
4. What's your outlook on life?
   1. Realistic
   2. Hopeful
   3. Cynical
   4. Romantic
   5. Adventurous
5. How emotional do you consider yourself?
   1. Very little
   2. Somewhat
   3. It varies
   4. A lot
   5. I wear my heart on my sleeve
Sentiment Score: 0.0
{'🎬 You are...': 'Bad Black', '📖 Description': 'A mild-mannered doctor is trained in the art of ass-kicking commando vengeance by a no-nonsense ghetto kid in an effort to regain a family heirloom from Uganda’s toughest gang.', '🎭 Genres': "['Action', 'Comedy', 'Crime']", '🌡 Sentiment Match Score': np.float64

some codes have been updated.